## Requirements
- Anaconda
- Python 3.5
- Pandas
- Requests

In [ ]:
import numpy as np
import pandas as pd
import requests

### Function
1. Loads addresses from a file pulled from excel file and cleans it for maps api
2. Generates request string
3. Receives json package in pandas dataframe that contains distance and duration from member address to potential location
4. Stores duration in dataframe and generates statistics on that data

In [ ]:
df = pd.read_excel('C:/Users/Chris/Documents/history_address.xlsx')
df = df[['Order Notes', 'Total']]
df = df[~df['Total'].isnull()]
df = df[~df['Total'].str.contains('Ship to')]
df['Total'] = df['Total'].str.replace(',', '').str.replace(' ', '+')
df['Total'].unique()

In [ ]:
destinations = ['1213 SW Adams Peoria IL',
                '201 Thunderbird Lane East Peoria IL',
                '201 Spring St Peoria IL',
                '7916 N Hale Peoria IL']
for i, destination in enumerate(destinations):
    destinations[i] = destination.replace(' ', '+')

In [ ]:
base_url = 'https://maps.googleapis.com/maps/api/distancematrix/json?'
units = 'imperial'
key = '#####################################'
response = []
for address in df['Total'].unique():
    column = []
    for destination in destinations:
        url = '{0}units={1}&origins={2}&destinations={3}&key={4}'.format(base_url, units, address, destination, key)
        res = requests.get(url)
        column.append(res)
    response.append(column)

In [ ]:
data = pd.DataFrame(response)
duration = data.applymap(lambda x: x.json())
duration.columns = destinations
duration.index = df['Total'].unique()
duration = duration[duration.applymap(lambda x: True if x['status'] == 'OK' else False)].dropna()
duration = duration[duration.applymap(lambda x: True if x['rows'][0]['elements'][0]['status'] == 'OK' else False)].dropna()
duration = duration.applymap(lambda x: x['rows'][0]['elements'][0]['duration']['value'])
duration = duration/60
duration.describe()